# DSFB Assignment 5

In this assignment, you will begin to work with text data and natural language processing. You will analyze aspects of th DonorsChoose.org program. Aspects of this project were first posed as a Kaggle challenge and the data comes from [Kaggle DonorsChoose.org Application Screening challenge](https://www.kaggle.com/c/donorschoose-application-screening/data). We have changed the nature of what you need to do in this assignment (so it does not track what was done in the Kaggle Challenge), but nevertheless using or referring to the Kaggle Challenge repository is not allowed for the assignment.

###  DonorsChoose.org  
  
Founded in 2000 by a high school teacher in the Bronx, DonorsChoose.org empowers public school teachers from across the country to request much-needed materials and experiences for their students. At any given time, there are thousands of classroom requests that can be brought to life with a gift of any amount. DonorsChoose.org receives hundreds of thousands of project proposals each year for classroom projects in need of funding. Right now, a large number of volunteers is needed to manually screen each submission before it's approved to be posted on the DonorsChoose.org website. In this assignment, you will analyze the text of the essays and requirements from each proposal.

<img src="https://cached.imagescaler.hbpl.co.uk/resize/scaleWidth/580/cached.offlinehbpl.hbpl.co.uk/news/NST/C8B9CC1D-03B0-9B80-4CFE78B5B539240F.jpg" width="500" height="500" align="center"/>

Image source: https://cached.imagescaler.hbpl.co.uk/resize/scaleWidth/580/cached.offlinehbpl.hbpl.co.uk/news/NST/C8B9CC1D-03B0-9B80-4CFE78B5B539240F.jpg

### Data

As you will see, this dataset includes many different kinds of features with structured and unstructured data. The dataset consists of application materials (see *application_data.csv*) and resources requested (see *resource_data.csv*). The application materials (see *application_data.csv*) contain the following features.

| Feature name  | Description  |
|----------------|--------------|
| id  | Unique id of the project application    |
| teacher_id    | id of the teacher submitting the application  |
| teacher_prefix    | title of the teacher's name (Ms., Mr., etc.)    |
| school_state    | US state of the teacher's school    |
| project_submitted_datetime    | application submission timestamp    |
| project_grade_category    | school grade levels (PreK-2, 3-5, 6-8, and 9-12)   |
| project_subject_categories   | category of the project (e.g., "Music & The Arts")    |
| project_subject_subcategories    | sub-category of the project (e.g., "Visual Arts")    |
| project_title    | title of the project    |
| project_essay_1    | first essay*   |
| project_essay_2    | second essay*    |
| project_essay_3    | third essay*   |
| project_essay_4    | fourth essay*  |
| project_resource_summary    | summary of the resources needed for the project    |
| teacher_number_of_previously_posted_projects   | number of previously posted applications by the submitting teacher    |
| project_is_approved    | whether DonorsChoose proposal was accepted (0="rejected", 1="accepted"); train.csv only    |


\*Note: Prior to May 17, 2016, the prompts for the essays were as follows:

  * project_essay_1: "Introduce us to your classroom"  

  * project_essay_2: "Tell us more about your students"  

  * project_essay_3: "Describe how your students will use the materials you're requesting"  

  * project_essay_4: "Close by sharing why your project will make a difference"  

Starting on May 17, 2016, the number of essays was reduced from 4 to 2, and the prompts for the first 2 essays were changed to the following:

  * project_essay_1: "Describe your students: What makes your students special? Specific details about their background, your neighborhood, and your school are all helpful."  

  * project_essay_2: "About your project: How will these materials make a difference in your students' learning and improve their school lives?"  

For all projects with project_submitted_datetime of 2016-05-17 and later, the values of project_essay_3 and project_essay_4 will be missing (i.e. NaN).


### Special NLP Libraries

We will use several new libraries for this assignment - so be sure to first install those on your machine by with `pip` in a terminal:

    pip install --user -U nltk
    pip install -U gensim
    pip install -U spacy
    pip install -U pyldavis

## IMPORTS

In [467]:
import importlib
# Standard imports
import numpy  as np
import pandas as pd

import itertools
import random
import math  
import copy

from pprint import pprint  # nicer printing

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Other NLP
import re
import spacy
import nltk
from nltk.corpus import stopwords

# General Plotting
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
%matplotlib inline  
import seaborn as sns
sns.set(style="white")

# Special Plotting
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

# ignore some warnings 
import warnings
warnings.filterwarnings('ignore')

# Set the maximum number of rows displayed by pandas
pd.options.display.max_rows = 1000

# Set some CONSTANTS that will be used later
SEED    = 41  # base to generate a random number
SCORE   = 'roc_auc'
FIGSIZE = (16, 10)

# PART 1: Prep

**PROBLEM**: To use a particular model in the `spacy` package, you need to manually download and install that particular model. You will need to run the following code from a terminal: `python -m spacy download en_core_web_sm`. Rather than doing that manually from bash in a separate terminal program, do it inline below using a "magic" command in jupyter. HINT: Use *!* followed by a bash command in a cell to run a bash command.

In [468]:
# Download en_core_web_sm for spacy

!python3 -m spacy download en_core_web_sm

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


**PROBLEM**: To confirm that `spacy` is working (and `en_core_web_sm` is installed on your computer), you should be able to use `spacy.load()` to build a `Language` object to perform some basic nlp. Do that below:

In [469]:
# Test use of spacy by using the spacy.load() function
import spacy
import en_core_web_sm
nlp = spacy.load('en_core_web_sm')

**PROBLEM**: Use nltk.download() to download a list of raw stopwords. (see NLTK documentation)

In [470]:
# Download NLTK stopwords

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ekaterinakryukova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**PROBLEM**: Use the `stopwords` object from `nltk` to build a list of English stopwords. 

In [471]:
# Get English Stopwords from NLTK
from nltk.corpus import stopwords
stopWords = stopwords.words('english')

In [472]:
print(len(stopWords))

179


**PROBLEM**: Extend your `stop_words` list with some additional stopwords that you believe should be ignored in this particular context.

In [473]:
# Extend the stop word list  

stopWords.extend(['from', 'subject', 're', 'edu', 'use'])

print(len(stopWords))

184


### Download the Data

Unlike other projects, this project includes a training set too big for GitHub. Through the terminal lab of Jupyter lab, download the data using the *wget* command, unzip it using the *zip* command and check that it's in the root directory of the project. 

Locations : 

    Applications dataset: https://storage.googleapis.com/dsfm-datasets/text-applications/application_data.csv.zip
    Resources dataset: https://storage.googleapis.com/dsfm-datasets/text-applications/resource_data.csv.zip
    
Hint: Use *wget* and *unzip* commands. Use *!* followed by a bash command in a cell to run a bash command.

**PROBLEM**: wget the data

In [474]:
# wget the data
import wget
wget.download('https://storage.googleapis.com/dsfm-datasets/text-applications/application_data.csv.zip','data')

'data/application_data.csv (1).zip'

In [475]:
wget.download('https://storage.googleapis.com/dsfm-datasets/text-applications/resource_data.csv.zip','data')

'data/resource_data.csv (1).zip'

**PROBLEM**: unzip the data

In [476]:
# unzip the data
from zipfile import ZipFile
zip = ZipFile('data/application_data.csv.zip')
zip.extractall('data/application_data')

In [477]:
zip = ZipFile('data/resource_data.csv.zip')
zip.extractall('data/resource_data')


# PART 2: Load Data

**PROBLEM**: Load `application_data.csv` and investigate it a bit.

In [478]:
# Load applications
application_data = pd.read_csv('data/application_data/application_data.csv',parse_dates=['project_submitted_datetime'])
application_data.head(5)


,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved
0,p036502,484aaf11257089a66cfedc9461c6bd0a,Ms.,NV,2016-11-18 14:45:59,Grades PreK-2,Literacy & Language,Literacy,Super Sight Word Centers,"Most of my kindergarten students come from low-income households and are considered \""at-risk\"". These kids walk to school alongside their parents and most have never been further than walking distance from their house. For 80% of my students, English is not their first language or the language spoken at home. \r\n\r\nWhile my kindergarten kids have many obstacles in front of them, they come to school each day excited and ready to learn. Most students started the year out never being in a school setting. At the start of the year many had never been exposed to letters. Each day they soak up more knowledge and try their hardest to succeed. They are highly motivated to learn new things every day. We are halfway through the year and they are starting to take off. They know know all letters, some sight words, numbers to 20, and a majority of their letter sounds because of their hard work and determination. I am excited to see the places we will go from here!",I currently have a differentiated sight word center that we do daily during our literacy stations. The students have activities that relate to whatever sight word list they are on. This is one of their favorite station activities. I want to continue to provide the students with engaging ways to practice their sight words. \r\n\r\nI dream of having the students use QR readers to scan the sight words that they are struggling with and the Ipods reading the sight words with them. This would help so many of my students by giving them multiple exposures to the words. My students need someone who can go over these sight words daily and I can't always get around to everyone to practice their flashcards with them. With the Ipods they would still have a way to practice their sight words on a daily basis.,NaN,NaN,My students need 6 Ipod Nano's to create and differentiated and engaging way to practice sight words during a literacy station.,26,1
1,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,2017-04-26 15:57:28,Grades 3-5,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",Keep Calm and Dance On,"Our elementary school is a culturally rich school, with a diverse population of 580 students, in Pre-K through sixth grade.\r\nOur Title I school population has 92% of students qualifying for free or reduced priced lunches and a high concentration of English Learners. We also serve two foster group homes for temporary and long-term placement of homeless children. \r\nWe do not see these statistics as road blocks. We see them as additions to our rich diversity. Together we will help students to develop to their fullest potential: Creative, problem-solving, compassionate adults.","We strive to provide our diverse population of students with not only extra curricular activities, but an outlet for them to express themselves creatively.As a teacher, I have organized a dance club for lower elementary that meets once a week after school.\r\n\r\n This gives the girls something to look forward to, fosters the education of the whole child, and creates a social environment for our varied cultural student body. \r\n\r\nSince beginning our dance club, I have watched our girls who normally are introverts, bloom with excitement. They are also choreographing dances and productions in content areas during the school day.",NaN,NaN,My students need matching shirts to wear for dance performances and competitions.,1,0
2,p233823,a9b876a9252e08a55e3d894150f75ba3,Ms.,UT,2017-01-01 22:57:44,Grades 3-5,"Math & Science, Literacy & Language","Applied Scien

In [479]:
application_data.shape

(182080, 16)

In [480]:
#type of date column
application_data.project_submitted_datetime.dtypes

dtype('<M8[ns]')

In [481]:
#number of nan values
application_data.project_essay_1.isna().sum(),application_data.project_essay_2.isna().sum(),application_data.project_essay_3.isna().sum(),application_data.project_essay_4.isna().sum(),

(0, 0, 175706, 175706)

**PROBLEM**: Load `resource_data.csv` and investigate it a bit.

In [482]:
# Load resources

resource_data = pd.read_csv('data/resource_data/resource_data.csv')
resource_data.head(5)

,id,description,quantity,price
0,p233245,LC652 - Lakeshore Double-Space Mobile Drying Rack,1,149.00
1,p069063,Bouncy Bands for Desks (Blue support pipes),3,14.95
2,p069063,Cory Stories: A Kid's Book About Living With Adhd,1,8.45
3,p069063,"Dixon Ticonderoga Wood-Cased #2 HB Pencils, Box of 96, Yellow (13872)",2,13.59
4,p069063,"EDUCATIONAL INSIGHTS FLUORESCENT LIGHT FILTERS (TRANQUIL BLUE), SET OF 4",3,24.95


**PROBLEM**: Some of the essays are NA. Replace NAs with empty strings.

In [483]:
# Replace NA values in essay columns with ''

application_data[['project_essay_3','project_essay_4']]=application_data[['project_essay_3',
                  'project_essay_4']].replace(np.nan, '', regex=True)

In [484]:
#count nan
application_data.project_essay_1.isna().sum(),application_data.project_essay_2.isna().sum(),application_data.project_essay_3.isna().sum(),application_data.project_essay_4.isna().sum(),

(0, 0, 0, 0)

In [485]:
application_data.columns

Index(['id', 'teacher_id', 'teacher_prefix', 'school_state',
       'project_submitted_datetime', 'project_grade_category',
       'project_subject_categories', 'project_subject_subcategories',
       'project_title', 'project_essay_1', 'project_essay_2',
       'project_essay_3', 'project_essay_4', 'project_resource_summary',
       'teacher_number_of_previously_posted_projects', 'project_is_approved'],
      dtype='object')

**PROBLEM**: To simplify matters, combine all essays into just one feature called "essays"

In [486]:
# Combine essays
application_data['essays']=application_data['project_essay_{}'.format(1)]
for i in range(2,5):
    application_data['essays']+=application_data['project_essay_{}'.format(i)].astype(str)

In [487]:
#get data with all essays
application_data[application_data.project_submitted_datetime<'2016-05-17'].head(1)

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,essays
18,p232007,e7a8f866e3174a77ffe37323f032a8ac,Mrs.,FL,2016-04-27 09:58:04,Grades PreK-2,"Applied Learning, Literacy & Language","College & Career Prep, Literature & Writing",Watch Readers Grow!,"During our reading workshop students are at daily 5. My students need activities to help them practice skills in a fun and enjoyable way that is on the level of each child. As the teacher I enjoy conferencing with each student, so the more engaged the students are practicing the skills they.",My students lack confidence. I have a class with such great potential. My students need more hands on learning and extra practice to catch and grow a love for reading. My second graders love to learn. We have resources but most are out dated. My students would be so bright if they could only build confidence. I believe in them and now I want to see them bloom.,"During our reading workshop. We do mini lessons that focus on a skill and then they rotate through daily 5 (centers). These reading activities will help them apply what they learned and reinforce the skills. They rotate through self read, buddy read (carpet) listening, writing, word work (reading skills). If the class is actively engaged this leave more time for small group learning and conferencing with students. The giant magnet words will help at word work to build sentences and use the correct parts of speech. The classroom carpet will be for read to self where students can cozy up with a good book. Read with a pen close reads will be used during buddy read where students can zoom into the meanings and what is being read. The language skill center and quick picks will be used for our writing center and will help students with the skills they need to be great writers. All these materials will help my students grow doing our daily readers workshop.",Your donations would greatly be a blessing for all my students. They focus on all the daily 5 rotations as well as help them grow. These reading activities will keep students engaged so the teacher can work and help students master the skills they are week in. The reading genre carpet will be used to enjoy a cozy place to sit and read. The close reads will be at buddy read so that students can help each other focus on what is being read. Thank you in advance for helping them grow as reader.,My students need these reading materials to help them learn in a fun and motivating way. With this reading genre carpet and reading and writing materials they will grow.,6,1,"During our reading workshop students are at daily 5. My students need activities to help them practice skills in a fun and enjoyable way that is on the level of each child. As the teacher I enjoy conferencing with each student, so the more engaged the students are practicing the skills they.My students lack confidence. I have a class with such great potential. My students need more hands on learning and extra practice to catch and grow a love for reading. My second graders love to learn. We have resources but most are out dated. My students would be so bright if they could only build confidence. I believe in them and now I want to see them bloom.During our reading workshop. We do mini lessons that focus on a skill and then they rotate through daily 5 (centers). These reading activities will help them apply what they learned and reinforce the skills. They rotate through self read, buddy read (carpet) listening, writing, word work (reading skills). If the class is actively engaged this leave more time for small group learning and conferencing with students. The giant magnet words will help at word work to build sentences and use the correct parts of speech. The classroom

In [488]:
#check
application_data['essays'][18]

'During our reading workshop students are at daily 5. My students need activities to help them practice skills in a fun and enjoyable way that is on the level of each child. As the teacher I enjoy conferencing with each student, so the more engaged the students are practicing the skills they.My students lack confidence. I have a class with such great potential. My students need more hands on learning and extra practice to catch and grow a love for reading. My second graders love to learn. We have resources but most are out dated. My students would be so bright if they could only build confidence. I believe in them and now I want to see them bloom.During our reading workshop. We do mini lessons that focus on a skill and then they rotate through daily 5 (centers). These reading activities will help them apply what they learned and reinforce the skills. They rotate through self read, buddy read (carpet) listening, writing,  word work (reading skills). If the class is actively engaged this

In [489]:
for i in range(1,5):
    print(application_data['project_essay_{}'.format(i)][18])

During our reading workshop students are at daily 5. My students need activities to help them practice skills in a fun and enjoyable way that is on the level of each child. As the teacher I enjoy conferencing with each student, so the more engaged the students are practicing the skills they.
My students lack confidence. I have a class with such great potential. My students need more hands on learning and extra practice to catch and grow a love for reading. My second graders love to learn. We have resources but most are out dated. My students would be so bright if they could only build confidence. I believe in them and now I want to see them bloom.
During our reading workshop. We do mini lessons that focus on a skill and then they rotate through daily 5 (centers). These reading activities will help them apply what they learned and reinforce the skills. They rotate through self read, buddy read (carpet) listening, writing,  word work (reading skills). If the class is actively engaged thi

In [490]:

#drop separate columns of essays
for i in range(1,5):
    application_data.drop(columns=['project_essay_{}'.format(i)],inplace=True)

In [491]:
application_data.columns

Index(['id', 'teacher_id', 'teacher_prefix', 'school_state',
       'project_submitted_datetime', 'project_grade_category',
       'project_subject_categories', 'project_subject_subcategories',
       'project_title', 'project_resource_summary',
       'teacher_number_of_previously_posted_projects', 'project_is_approved',
       'essays'],
      dtype='object')

In [492]:
application_data.columns

Index(['id', 'teacher_id', 'teacher_prefix', 'school_state',
       'project_submitted_datetime', 'project_grade_category',
       'project_subject_categories', 'project_subject_subcategories',
       'project_title', 'project_resource_summary',
       'teacher_number_of_previously_posted_projects', 'project_is_approved',
       'essays'],
      dtype='object')

In [493]:
application_data.shape

(182080, 13)

In [494]:
resource_data.head()

,id,description,quantity,price
0,p233245,LC652 - Lakeshore Double-Space Mobile Drying Rack,1,149.00
1,p069063,Bouncy Bands for Desks (Blue support pipes),3,14.95
2,p069063,Cory Stories: A Kid's Book About Living With Adhd,1,8.45
3,p069063,"Dixon Ticonderoga Wood-Cased #2 HB Pencils, Box of 96, Yellow (13872)",2,13.59
4,p069063,"EDUCATIONAL INSIGHTS FLUORESCENT LIGHT FILTERS (TRANQUIL BLUE), SET OF 4",3,24.95


**PROBLEM**: Merge the resources and application datasets on the *id* feature.

In [495]:
resource_data=resource_data.fillna(' ')
resource_data=resource_data.drop_duplicates()
resource_data['description']=resource_data.groupby(['id'])['description'].transform(lambda x : ' '.join(x)) 


In [496]:
# Merge two datasets


merged_df=application_data.merge(resource_data, on='id', how='left')
# Check the data to confirm it worked

merged_df.columns


Index(['id', 'teacher_id', 'teacher_prefix', 'school_state',
       'project_submitted_datetime', 'project_grade_category',
       'project_subject_categories', 'project_subject_subcategories',
       'project_title', 'project_resource_summary',
       'teacher_number_of_previously_posted_projects', 'project_is_approved',
       'essays', 'description', 'quantity', 'price'],
      dtype='object')

In [497]:
merged_df.shape

(1073254, 16)

In [498]:
resource_data.shape,resource_data.id.nunique()

((1528928, 4), 260115)

In [499]:
application_data.shape

(182080, 13)

In [500]:
application_data.id.nunique()

182080

**PROBLEM**: Keep the following data for additional analysis (the id and the text features): `id`, `school_state`, `project_subject_categories`, `project_subject_subcategories`, `essays`, `description`

In [501]:
FEATURE_NAMES = ['school_state', 'project_subject_categories', 'project_subject_subcategories', 'essays', 'description']

In [502]:
# Keep the Text Featuresss

merged_df_textual=merged_df[['id']+FEATURE_NAMES]

In [503]:
merged_df_textual.head()

,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
0,p036502,NV,Literacy & Language,Literacy,"Most of my kindergarten students come from low-income households and are considered \""at-risk\"". These kids walk to school alongside their parents and most have never been further than walking distance from their house. For 80% of my students, English is not their first language or the language spoken at home. \r\n\r\nWhile my kindergarten kids have many obstacles in front of them, they come to school each day excited and ready to learn. Most students started the year out never being in a school setting. At the start of the year many had never been exposed to letters. Each day they soak up more knowledge and try their hardest to succeed. They are highly motivated to learn new things every day. We are halfway through the year and they are starting to take off. They know know all letters, some sight words, numbers to 20, and a majority of their letter sounds because of their hard work and determination. I am excited to see the places we will go from here!I currently have a differentiated sight word center that we do daily during our literacy stations. The students have activities that relate to whatever sight word list they are on. This is one of their favorite station activities. I want to continue to provide the students with engaging ways to practice their sight words. \r\n\r\nI dream of having the students use QR readers to scan the sight words that they are struggling with and the Ipods reading the sight words with them. This would help so many of my students by giving them multiple exposures to the words. My students need someone who can go over these sight words daily and I can't always get around to everyone to practice their flashcards with them. With the Ipods they would still have a way to practice their sight words on a daily basis.",Apple - iPod nano� 16GB MP3 Player (8th Generation - Latest Model) - Blue Apple - iPod nano� 16GB MP3 Player (8th Generation - Latest Model) - Silver
1,p036502,NV,Literacy & Language,Literacy,"Most of my kindergarten students come from low-income households and are considered \""at-risk\"". These kids walk to school alongside their parents and most have never been further than walking distance from their house. For 80% of my students, English is not their first language or the language spoken at home. \r\n\r\nWhile my kindergarten kids have many obstacles in front of them, they come to school each day excited and ready to learn. Most students started the year out never being in a school setting. At the start of the year many had never been exposed to letters. Each day they soak up more knowledge and try their hardest to succeed. They are highly motivated to learn new things every day. We are halfway through the year and they are starting to take off. They know know all letters, some sight words, numbers to 20, and a majority of their letter sounds because of their hard work and determination. I am excited to see the places we will go from here!I currently have a differentiated sight word center that we do daily during our literacy stations. The students have activities that relate to whatever sight word list they are on. This is one of their favorite station activities. I want to continue to provide the students with engaging ways to practice their sight words. \r\n\r\nI dream of having the students use QR readers to scan the sight words that they are struggling with and the Ipods reading the sight words with them. This would help so many of my students by giving them multiple exposures to the words. My students need someone who can go over these sight words daily and I can't always get around to everyone to practice their flashcards with them. With the Ipods they would still have a way to practice their sight words on a daily basis.",Apple - iPod nano� 16GB MP3 Player (8th Generation - Latest Model) - Blue Apple - iPod nano� 16GB MP3 Player (8th Gener

In [504]:
merged_df_textual=merged_df_textual.drop_duplicates()

In [505]:
merged_df_textual.shape

(182080, 6)

In [506]:
#merged_df_textual=merged_df_textual.sample(n=100000, random_state=1)
merged_df_textual.to_csv('merged_df_textual.csv',index=False)

# PART 3: Preprocess Text

Make an independent copy of the data so we can restart here when testing...

In [507]:
data = copy.copy(merged_df_textual).fillna(' ')  # when "merged" is the pandas dataframe

**PROBLEM**: Define a custom function `clean_punctuation()` to remove some punctuation from your text data. You don't have to do absolutely everything one might want to do - just show that you can do it. Start with each some easy operations with `str.replace()`.

In [508]:
# Define a custom function to clean punctuation from  given text

def clean_punctuation(txt):
    txt=txt.replace('&', ' ')
    txt=txt.replace('.', ' ')
    txt=txt.replace("\\r\\n", " ")
    return txt

**PROBLEM**: Use the `apply()` function from pandas to _apply_ that function down the `essays` column of your data.

In [509]:
# Apply your function to clean the essays column
for feature in ['essays']:
    data[feature]=data[feature].apply(clean_punctuation)
    
    
    
data.head()

,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
0,p036502,NV,Literacy & Language,Literacy,"Most of my kindergarten students come from low-income households and are considered \""at-risk\"" These kids walk to school alongside their parents and most have never been further than walking distance from their house For 80% of my students, English is not their first language or the language spoken at home While my kindergarten kids have many obstacles in front of them, they come to school each day excited and ready to learn Most students started the year out never being in a school setting At the start of the year many had never been exposed to letters Each day they soak up more knowledge and try their hardest to succeed They are highly motivated to learn new things every day We are halfway through the year and they are starting to take off They know know all letters, some sight words, numbers to 20, and a majority of their letter sounds because of their hard work and determination I am excited to see the places we will go from here!I currently have a differentiated sight word center that we do daily during our literacy stations The students have activities that relate to whatever sight word list they are on This is one of their favorite station activities I want to continue to provide the students with engaging ways to practice their sight words I dream of having the students use QR readers to scan the sight words that they are struggling with and the Ipods reading the sight words with them This would help so many of my students by giving them multiple exposures to the words My students need someone who can go over these sight words daily and I can't always get around to everyone to practice their flashcards with them With the Ipods they would still have a way to practice their sight words on a daily basis",Apple - iPod nano� 16GB MP3 Player (8th Generation - Latest Model) - Blue Apple - iPod nano� 16GB MP3 Player (8th Generation - Latest Model) - Silver
2,p039565,GA,"Music & The Arts, Health & Sports","Performing Arts, Team Sports","Our elementary school is a culturally rich school, with a diverse population of 580 students, in Pre-K through sixth grade Our Title I school population has 92% of students qualifying for free or reduced priced lunches and a high concentration of English Learners We also serve two foster group homes for temporary and long-term placement of homeless children We do not see these statistics as road blocks We see them as additions to our rich diversity Together we will help students to develop to their fullest potential: Creative, problem-solving, compassionate adults We strive to provide our diverse population of students with not only extra curricular activities, but an outlet for them to express themselves creatively As a teacher, I have organized a dance club for lower elementary that meets once a week after school This gives the girls something to look forward to, fosters the education of the whole child, and creates a social environment for our varied cultural student body Since beginning our dance club, I have watched our girls who normally are introverts, bloom with excitement They are also choreographing dances and productions in content areas during the school day",Reebok Girls' Fashion Dance Graphic T-Shirt - Dd Dark Heather Grey - L
3,p233823,UT,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",Hello; My name is Mrs Brotherton I teach 5th grade at Ascent Academy in Utah We are a wonderful charter school that uses the students' interests to help them learn We are always looking for wonderful teaching methods to help students My students are wonderful I have several levels in my class Our school is big on curriculum compacting and helping gifted kids move faster and struggling kids get the extra help they need We would benefit so much from your donation Every little bit helps us reach our goal I teach Science to all 88 5th g

**PROBLEM**: Define **another** custom function called `clean_re()` to clean your text data using regular expressions. Do at least two "cleanings" (i.e., show that you can use the `re` library).

In [510]:
# Define a custom function to clean some given text
import re

def clean_re(txt):
    p = re.compile(r'[^\w\s]')
    txt=p.sub('', txt)
    
    return txt

In [511]:
# Apply clean_re() to all features
from tqdm import tqdm
for feature in tqdm(FEATURE_NAMES):
    data[feature]=data[feature].fillna('').astype(str).apply(clean_re)
    
    
data.head()


100%|██████████| 5/5 [00:10<00:00,  2.16s/it]


,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
0,p036502,NV,Literacy Language,Literacy,Most of my kindergarten students come from lowincome households and are considered atrisk These kids walk to school alongside their parents and most have never been further than walking distance from their house For 80 of my students English is not their first language or the language spoken at home While my kindergarten kids have many obstacles in front of them they come to school each day excited and ready to learn Most students started the year out never being in a school setting At the start of the year many had never been exposed to letters Each day they soak up more knowledge and try their hardest to succeed They are highly motivated to learn new things every day We are halfway through the year and they are starting to take off They know know all letters some sight words numbers to 20 and a majority of their letter sounds because of their hard work and determination I am excited to see the places we will go from hereI currently have a differentiated sight word center that we do daily during our literacy stations The students have activities that relate to whatever sight word list they are on This is one of their favorite station activities I want to continue to provide the students with engaging ways to practice their sight words I dream of having the students use QR readers to scan the sight words that they are struggling with and the Ipods reading the sight words with them This would help so many of my students by giving them multiple exposures to the words My students need someone who can go over these sight words daily and I cant always get around to everyone to practice their flashcards with them With the Ipods they would still have a way to practice their sight words on a daily basis,Apple iPod nano 16GB MP3 Player 8th Generation Latest Model Blue Apple iPod nano 16GB MP3 Player 8th Generation Latest Model Silver
2,p039565,GA,Music The Arts Health Sports,Performing Arts Team Sports,Our elementary school is a culturally rich school with a diverse population of 580 students in PreK through sixth grade Our Title I school population has 92 of students qualifying for free or reduced priced lunches and a high concentration of English Learners We also serve two foster group homes for temporary and longterm placement of homeless children We do not see these statistics as road blocks We see them as additions to our rich diversity Together we will help students to develop to their fullest potential Creative problemsolving compassionate adults We strive to provide our diverse population of students with not only extra curricular activities but an outlet for them to express themselves creatively As a teacher I have organized a dance club for lower elementary that meets once a week after school This gives the girls something to look forward to fosters the education of the whole child and creates a social environment for our varied cultural student body Since beginning our dance club I have watched our girls who normally are introverts bloom with excitement They are also choreographing dances and productions in content areas during the school day,Reebok Girls Fashion Dance Graphic TShirt Dd Dark Heather Grey L
3,p233823,UT,Math Science Literacy Language,Applied Sciences Literature Writing,Hello My name is Mrs Brotherton I teach 5th grade at Ascent Academy in Utah We are a wonderful charter school that uses the students interests to help them learn We are always looking for wonderful teaching methods to help students My students are wonderful I have several levels in my class Our school is big on curriculum compacting and helping gifted kids move faster and struggling kids get the extra help they need We would benefit so much from your donation Every little bit helps us reach our goal I teach Science to all 88 5th grade students and have clusters of 25 new students every 8 weeks Your donation will h

In [512]:
data.to_csv('clean_re.csv', index=False)    

data.shape

(182080, 6)

In [513]:
data=pd.read_csv('clean_re.csv',lineterminator='\n')

In [514]:
data.shape


(182080, 6)

In [515]:
data['description'].head(10)

0    Apple  iPod nano 16GB MP3 Player 8th Generation  Latest Model  Blue Apple  iPod nano 16GB MP3 Player 8th Generation  Latest Model  Silver                                           
1    Reebok Girls Fashion Dance Graphic TShirt  Dd Dark Heather Grey  L                                                                                                                  
2    3doodler Start Full Edu Bundle                                                                                                                                                      
3    BALL PG 4 POLY SET OF 6 COLORS BALL PLAYGROUND POLY 85 SET OF 6 KIT JUMBO GRADESTUFF PACK PARACHUTE GRIPSTARCHUTE 24 RECESS PACK GRADE K VIOLET                                     
4    Crown Berkey Water Filter With 2 Black and 2 PF2 Fluoride Filters                                                                                                                   
5    Amazon  Fire Kids Edition  7 Tablet  16GB  Green Amazon  Fire Kid

In [516]:
data.drop_duplicates().shape

(182080, 6)

**PROBLEM**: Remove stopwords. (Hint: use stopwords from nltk's `stopwords()` plus any additions you'd like to make. Then, again, define a custom function and then apply it to all features.)

In [517]:
stopWords.extend(['although','engaging','approximately','yet','nan','u','us','would','would','see','big','student','school','many'])


In [518]:
# Define custom function to remove stopwords
df = copy.copy(data)  
df=df.drop_duplicates()
def clean_stopword(txt):
    txt=txt.lower().split()
    filtered_sentence = [w for w in txt if  w  not in  stopWords]  
    filtered_sentence=' '.join(filtered_sentence)
    return filtered_sentence

In [519]:
# Apply function to remove stopwords  
from tqdm import tqdm
for feature in tqdm(FEATURE_NAMES):
    df[feature]=df[feature].fillna(' ').apply(clean_stopword)
    
    
df.to_csv('clean_stopword.csv', index=False)       
df.head(10)



100%|██████████| 5/5 [1:27:06<00:00, 1045.36s/it]


,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
0,p036502,nv,literacy language,literacy,kindergarten students come lowincome households considered atrisk kids walk alongside parents never walking distance house 80 students english first language language spoken home kindergarten kids obstacles front come day excited ready learn students started year never setting start year never exposed letters day soak knowledge try hardest succeed highly motivated learn new things every day halfway year starting take know know letters sight words numbers 20 majority letter sounds hard work determination excited places go herei currently differentiated sight word center daily literacy stations students activities relate whatever sight word list one favorite station activities want continue provide students ways practice sight words dream students qr readers scan sight words struggling ipods reading sight words help students giving multiple exposures words students need someone go sight words daily cant always get around everyone practice flashcards ipods still way practice sight words daily basis,apple ipod nano 16gb mp3 player 8th generation latest model blue apple ipod nano 16gb mp3 player 8th generation latest model silver
1,p039565,ga,music arts health sports,performing arts team sports,elementary culturally rich diverse population 580 students prek sixth grade title population 92 students qualifying free reduced priced lunches high concentration english learners also serve two foster group homes temporary longterm placement homeless children statistics road blocks additions rich diversity together help students develop fullest potential creative problemsolving compassionate adults strive provide diverse population students extra curricular activities outlet express creatively teacher organized dance club lower elementary meets week gives girls something look forward fosters education whole child creates social environment varied cultural body since beginning dance club watched girls normally introverts bloom excitement also choreographing dances productions content areas day,reebok girls fashion dance graphic tshirt dd dark heather grey l
2,p233823,ut,math science literacy language,applied sciences literature writing,hello name mrs brotherton teach 5th grade ascent academy utah wonderful charter uses students interests help learn always looking wonderful teaching methods help students students wonderful several levels class curriculum compacting helping gifted kids move faster struggling kids get extra help need benefit much donation every little bit helps reach goal teach science 88 5th grade students clusters 25 new students every 8 weeks donation help one 5th grade class help several students mention students years come looking add 3doodler classroom wonderful set 3doodler order help students achieve mission using technology help students express learned medium choice 3doodler class fast advanced learners able go ahead start project help students get 3doodler also allow one students find medium help learn retain knowledge,3doodler start full bundle
3,p185307,nc,health sports,health wellness,students greatest students socially economically disadvantaged inner city limited activities pe physical education inside violence location located physical activities students active within good program students either african american hispanic students range enrolled kindergarten8th grade since physical education important ones success students pe class monday friday every day proper equipment pe always possible asking help shelving project kidinspired want better fitness look back pe class said 30 minutes day go extended day yearround 300 afternoon 30 minutes physical activity said need equipment active students kid inspired want variety equipment life changing physical activity students project totally kidinspireddecided needed equipment keep active gaining better health kidinspired realized needed 

In [520]:
df=pd.read_csv('clean_stopword.csv',lineterminator='\n')

In [521]:
df[-10:]

,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
182070,p068185,ok,literacy language math science,literature writing mathematics,hamilton elementary title 1 recieves 100 free breakfast lunch students things life working want make sure teacher students afforded opportunities educational equity students access upper grade classrooms switching self paced learning platform called summit program allows students set goals every week oneonone mentor work small group settings recieve individual attention regular gen ed classroom gives students power choose time place pace path want make sure teacher students afforded opportunities educational equity students access resources used classroom students freedom choose place work comfortable working current desks plastic hard chairs cracking edges definitely ideal 9 hour extended day new comfortable seating arrangements students able work comfort group projects individualized learning small teacherled groups please support hamilton husky family make dream kiddos reality anything friend donate help greatly,joe dorm chair limo black inflated stability wobble cushion including free pump exercise fitness core balance discbluesize 13 inches 33 cm diameter norwood commercial furniture nor1101acso plastic stack stools 1775 height 1175 width 1175 length assorted pack 5 peace yoga zafu meditation yoga buckwheat filled round cotton bolster pillow cushion blue 16 x 16
182071,p248714,tn,literacy language math science,literacy mathematics,first grade students come everyday excited ready learn students classroom come variety backgrounds fortunate enough travel world others fortunate regardless students background administrators teachers parents volunteers whatever takes make sure children needs met known community district going beyond expectations meet needs students walked students reading books playing instruments acting dancing drawing beautiful pictures schools mission create culturally diverse tradition excellence students encouraged excel academically learning skills necessary responsible confident lifelong learners ad productive members everchanging society order make classroom inviting environment give students space need focus learning must rug classroom need rug morning meeting whole group lessons calendar time rug classroom allow students stay focused learning common place meet share ideas new learning storage drawers post charts markers post notes educational books enhance learning literacy math stations small group lessons,number sense routines building numerical literacy every day grades k3 ottomanson jenny collection blue base multi colors kids childrens educational solar system design area classroom rugs 710 x 910 dark red postit selfstick easel pad 25 x 305 inches 30sheet pad 2 pack postit super sticky notes 4 x 4 rio de janeiro collection lined 6 padspack 6756ssuc sharpie flip chart markers assorted colors box 8 22478 sterilite clearview 3 storage drawer organizer teaching common core reading standards literature 2
182072,p045565,tx,applied learning literacy language,early development literature writing,blessed work rural quickly growing town north houston texas kinderbabies come variety backgrounds living lakeside fancy houses sharing beds siblings old trailers title 1 campus sets high expectations students academically personally desire give experiences opportunities succeed want push achieve goals develop selfmotivated learners fun learning goal teach students independent learning individuals items allow teach independence writing organization promoting writing process developing kinderbabies confident students students able get materials quickly easily minimizing time spend passing allowing spend time working students oneonone developing skills kindergarten students confident set successful years,dg273 heavyduty shelves cubbies storage unit jj926 classic birch tabletop writing center lm113 heavyduty bins set 3 ra414 tabletop paper center 

In [522]:
df.shape

(182080, 6)

**PROBLEM**: Now use Gensim’s `simple_preprocess()` function to tokenize and clean up your text data. TIP: `simple_preprocess()` returns a list of words, so we want to wrap it with a function that joins the list back together into a string.

In [523]:
# Define custom function to wrap c from gensim
from gensim.utils import simple_preprocess
df2 = copy.copy(df) 
def simple_preprocess_custom(txt):
    txt=simple_preprocess(txt,deacc=True)
    txt=' '.join(txt)
    return txt

In [524]:
# Apply simple_preprocess() to all features

for feature in tqdm(FEATURE_NAMES):
    df2[feature]=df2[feature].fillna('').apply(simple_preprocess_custom)

100%|██████████| 5/5 [36:12<00:00, 434.54s/it]


In [525]:
df2.head()
df2.to_csv('simple_preprocess_custom.csv', index=False)       


In [526]:
df2=pd.read_csv('simple_preprocess_custom.csv',lineterminator='\n')


In [527]:
df2.head()

,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
0,p036502,nv,literacy language,literacy,kindergarten students come lowincome households considered atrisk kids walk alongside parents never walking distance house students english first language language spoken home kindergarten kids obstacles front come day excited ready learn students started year never setting start year never exposed letters day soak knowledge try hardest succeed highly motivated learn new things every day halfway year starting take know know letters sight words numbers majority letter sounds hard work determination excited places go herei currently differentiated sight word center daily literacy stations students activities relate whatever sight word list one favorite station activities want continue provide students ways practice sight words dream students qr readers scan sight words struggling ipods reading sight words help students giving multiple exposures words students need someone go sight words daily cant always get around everyone practice flashcards ipods still way practice sight words daily basis,apple ipod nano gb mp player th generation latest model blue apple ipod nano gb mp player th generation latest model silver
1,p039565,ga,music arts health sports,performing arts team sports,elementary culturally rich diverse population students prek sixth grade title population students qualifying free reduced priced lunches high concentration english learners also serve two foster group homes temporary longterm placement homeless children statistics road blocks additions rich diversity together help students develop fullest potential creative problemsolving compassionate adults strive provide diverse population students extra curricular activities outlet express creatively teacher organized dance club lower elementary meets week gives girls something look forward fosters education whole child creates social environment varied cultural body since beginning dance club watched girls normally introverts bloom excitement also choreographing dances productions content areas day,reebok girls fashion dance graphic tshirt dd dark heather grey
2,p233823,ut,math science literacy language,applied sciences literature writing,hello name mrs brotherton teach th grade ascent academy utah wonderful charter uses students interests help learn always looking wonderful teaching methods help students students wonderful several levels class curriculum compacting helping gifted kids move faster struggling kids get extra help need benefit much donation every little bit helps reach goal teach science th grade students clusters new students every weeks donation help one th grade class help several students mention students years come looking add doodler classroom wonderful set doodler order help students achieve mission using technology help students express learned medium choice doodler class fast advanced learners able go ahead start project help students get doodler also allow one students find medium help learn retain knowledge,doodler start full bundle
3,p185307,nc,health sports,health wellness,students greatest students socially economically disadvantaged inner city limited activities pe physical education inside violence location located physical activities students active within good program students either african american hispanic students range enrolled kindergarten th grade since physical education important ones success students pe class monday friday every day proper equipment pe always possible asking help shelving project kidinspired want better fitness look back pe class said minutes day go extended day yearround afternoon minutes physical activity said need equipment active students kid inspired want variety equipment life changing physical activity students project totally needed equipment keep active gaining better health kidinspired realized needed minutes physical activity going make happen get minutes pe daily com

In [528]:
df2.shape

(182080, 6)

**PROBLEM**: Lemmatize the text. (Hint: Define a custom function and then apply it to all features.)

In [529]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ekaterinakryukova/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [530]:
importlib.reload(nltk)
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from collections import Counter

In [531]:
df3=copy.copy(df2)



In [532]:
df3.head(1)

,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
0,p036502,nv,literacy language,literacy,kindergarten students come lowincome households considered atrisk kids walk alongside parents never walking distance house students english first language language spoken home kindergarten kids obstacles front come day excited ready learn students started year never setting start year never exposed letters day soak knowledge try hardest succeed highly motivated learn new things every day halfway year starting take know know letters sight words numbers majority letter sounds hard work determination excited places go herei currently differentiated sight word center daily literacy stations students activities relate whatever sight word list one favorite station activities want continue provide students ways practice sight words dream students qr readers scan sight words struggling ipods reading sight words help students giving multiple exposures words students need someone go sight words daily cant always get around everyone practice flashcards ipods still way practice sight words daily basis,apple ipod nano gb mp player th generation latest model blue apple ipod nano gb mp player th generation latest model silver


In [533]:
# Write a lemmatization function based on nltk.stem.WordNetLemmatizer() -didn't complete for night


In [534]:
def get_pos( word ):
    w_synsets = wordnet.synsets(word)

    pos_counts = Counter()
    pos_counts["n"] = len(  [ item for item in w_synsets if item.pos()=="n"]  )
    pos_counts["v"] = len(  [ item for item in w_synsets if item.pos()=="v"]  )
    pos_counts["a"] = len(  [ item for item in w_synsets if item.pos()=="a"]  )
    pos_counts["r"] = len(  [ item for item in w_synsets if item.pos()=="r"]  )
    
    most_common_pos_list = pos_counts.most_common(3)
    return most_common_pos_list[0][0]

In [535]:

wnl = WordNetLemmatizer()
w_tokenizer = WhitespaceTokenizer()
def lemmatize_text(text):
    
    #lemmatize = lru_cache(maxsize=50000)(wnl.lemmatize)
    text=[wnl.lemmatize(w,get_pos(w)) for w in w_tokenizer.tokenize(text)]
    return text

In [536]:
# Apply lemmatize_text() to all features  
#As it is impossible to process all data, i will process 500 000
from tqdm import tqdm


from tqdm._tqdm_notebook import tqdm_notebook

for feature in tqdm(['essays', 'description']):
    tqdm_notebook.pandas()
    df3[feature]=df3[feature].fillna(' ').progress_apply(lemmatize_text)


  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [6:45:45<6:45:45, 24345.58s/it]

100%|██████████| 2/2 [6:50:34<00:00, 12317.49s/it]  

In [537]:
df3.head(1)

,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
0,p036502,nv,literacy language,literacy,"[kindergarten, student, come, lowincome, household, consider, atrisk, kid, walk, alongside, parent, never, walk, distance, house, student, english, first, language, language, speak, home, kindergarten, kid, obstacle, front, come, day, excite, ready, learn, student, start, year, never, set, start, year, never, expose, letter, day, soak, knowledge, try, hard, succeed, highly, motivate, learn, new, thing, every, day, halfway, year, start, take, know, know, letter, sight, word, number, majority, letter, sound, hard, work, determination, excite, place, go, herei, currently, differentiate, sight, word, center, daily, literacy, station, student, activity, relate, whatever, sight, word, list, one, favorite, station, activity, want, continue, provide, student, way, practice, sight, ...]","[apple, ipod, nano, gb, mp, player, th, generation, late, model, blue, apple, ipod, nano, gb, mp, player, th, generation, late, model, silver]"


In [538]:
df3.head(1)

,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
0,p036502,nv,literacy language,literacy,"[kindergarten, student, come, lowincome, household, consider, atrisk, kid, walk, alongside, parent, never, walk, distance, house, student, english, first, language, language, speak, home, kindergarten, kid, obstacle, front, come, day, excite, ready, learn, student, start, year, never, set, start, year, never, expose, letter, day, soak, knowledge, try, hard, succeed, highly, motivate, learn, new, thing, every, day, halfway, year, start, take, know, know, letter, sight, word, number, majority, letter, sound, hard, work, determination, excite, place, go, herei, currently, differentiate, sight, word, center, daily, literacy, station, student, activity, relate, whatever, sight, word, list, one, favorite, station, activity, want, continue, provide, student, way, practice, sight, ...]","[apple, ipod, nano, gb, mp, player, th, generation, late, model, blue, apple, ipod, nano, gb, mp, player, th, generation, late, model, silver]"


In [539]:
df33=copy.copy(df3)

In [540]:
stopWords.extend(['learn', 'classroom','help','need','work','th','come','class','love','able','year','time','want','make'])


In [565]:
stopWords.extend(['also'])

In [571]:
stopWords.extend(['get','day','whatever'])

In [572]:
#AFTER LEMMATIZATION IT IS NEEDED TO REMOVE STTOPWORDS
from tqdm import tqdm
for feature in tqdm(['essays', 'description']):
    df33[feature]=df33[feature].fillna(' ').apply(lambda w: [i for i in w if  i  not in  stopWords])

100%|██████████| 2/2 [01:21<00:00, 40.89s/it]


In [542]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
df33.head(1)

,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
0,p036502,nv,literacy language,literacy,"[kindergarten, lowincome, household, consider, atrisk, kid, walk, alongside, parent, never, walk, distance, house, english, first, language, language, speak, home, kindergarten, kid, obstacle, front, day, excite, ready, start, never, set, start, never, expose, letter, day, soak, knowledge, try, hard, succeed, highly, motivate, new, thing, every, day, halfway, start, take, know, know, letter, sight, word, number, majority, letter, sound, hard, determination, excite, place, go, herei, currently, differentiate, sight, word, center, daily, literacy, station, activity, relate, whatever, sight, word, list, one, favorite, station, activity, continue, provide, way, practice, sight, word, dream, qr, reader, scan, sight, word, struggle, ipod, read, sight, word, give, multiple, ...]","[apple, ipod, nano, gb, mp, player, generation, late, model, blue, apple, ipod, nano, gb, mp, player, generation, late, model, silver]"


In [543]:
df33.to_csv('lemmatize_text.csv', index=False)  

**PROBLEM**: What happened to the data in the pandas dataframe>

ANSWER: It was converted from long text into a list of individual words.

# PART 4:  Make an LDA topic model for the ESSAYS.

**Note: Part 4 is worth 10 points (the value of 10 individual problems).**

Define an LDA topic model for the `essays`. Compute the "Coherence score." Visually inspect the topic model by inspecting the top keywords from each model. Gensim provides functions for all of these tasks.  

In [573]:

# Create Dictionary
id2word = corpora.Dictionary( df33['essays'])

# Create Corpus
texts =  df33['essays']
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View unique id for each word in the essay
print(corpus[:1])


[[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 3), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 1), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 2), (36, 2), (37, 2), (38, 2), (39, 1), (40, 2), (41, 3), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 3), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 3), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 8), (65, 1), (66, 1), (67, 1), (68, 1), (69, 3), (70, 2), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 2), (78, 2), (79, 1), (80, 9)]]


In [574]:
# View word a given id corresponds to for id=0
id2word[0]

'activity'

In [575]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('activity', 2),
  ('alongside', 1),
  ('always', 1),
  ('around', 1),
  ('atrisk', 1),
  ('basis', 1),
  ('cant', 1),
  ('center', 1),
  ('consider', 1),
  ('continue', 1),
  ('currently', 1),
  ('daily', 3),
  ('determination', 1),
  ('differentiate', 1),
  ('distance', 1),
  ('dream', 1),
  ('english', 1),
  ('every', 1),
  ('everyone', 1),
  ('excite', 2),
  ('expose', 1),
  ('exposure', 1),
  ('favorite', 1),
  ('first', 1),
  ('flashcard', 1),
  ('front', 1),
  ('give', 1),
  ('go', 2),
  ('halfway', 1),
  ('hard', 2),
  ('herei', 1),
  ('highly', 1),
  ('home', 1),
  ('house', 1),
  ('household', 1),
  ('ipod', 2),
  ('kid', 2),
  ('kindergarten', 2),
  ('know', 2),
  ('knowledge', 1),
  ('language', 2),
  ('letter', 3),
  ('list', 1),
  ('literacy', 1),
  ('lowincome', 1),
  ('majority', 1),
  ('motivate', 1),
  ('multiple', 1),
  ('never', 3),
  ('new', 1),
  ('number', 1),
  ('obstacle', 1),
  ('one', 1),
  ('parent', 1),
  ('place', 1),
  ('practice', 3),
  ('provide', 1),

In [613]:
#I have tried several topics -> the best coherence score is for 10
#freeze_support()
    ##
    ##z Build Multicore LDA
lda_multicore_model = gensim.models.ldamulticore.LdaMulticore(corpus, num_topics = 10, id2word=id2word,random_state=100,
                                           passes=2, workers=2)
    # Saving trained model
    #lda_multicore_model.save('LDA_NYT_multicore')


In [607]:
lda_multicore_model.save('LDA_NYT_multicore8')

In [608]:
# Loading trained model
lda_multicore_model = gensim.models.ldamodel.LdaModel.load('LDA_NYT_multicore8')


In [614]:
# Print the Keyword in the 7 topics
print(lda_multicore_model.print_topics())
doc_lda = lda_multicore_model[corpus]

[(0, '0.028*"art" + 0.026*"language" + 0.018*"child" + 0.017*"english" + 0.013*"create" + 0.013*"material" + 0.013*"write" + 0.012*"kindergarten" + 0.010*"play" + 0.010*"new"'), (1, '0.033*"math" + 0.017*"skill" + 0.017*"read" + 0.017*"group" + 0.012*"grade" + 0.011*"center" + 0.011*"activity" + 0.011*"material" + 0.010*"game" + 0.010*"practice"'), (2, '0.044*"technology" + 0.018*"computer" + 0.017*"access" + 0.009*"chromebooks" + 0.009*"allow" + 0.009*"ipads" + 0.008*"project" + 0.008*"research" + 0.008*"ipad" + 0.008*"opportunity"'), (3, '0.023*"seat" + 0.018*"sit" + 0.016*"move" + 0.015*"chair" + 0.013*"focus" + 0.012*"allow" + 0.009*"desk" + 0.008*"best" + 0.008*"option" + 0.008*"provide"'), (4, '0.014*"supply" + 0.008*"home" + 0.008*"one" + 0.007*"family" + 0.007*"project" + 0.007*"teacher" + 0.007*"take" + 0.006*"every" + 0.006*"parent" + 0.006*"kid"'), (5, '0.018*"skill" + 0.012*"education" + 0.010*"disability" + 0.010*"special" + 0.010*"academic" + 0.009*"high" + 0.008*"support

In [615]:
# Compute Perplexity
#print('\nPerplexity: ', lda_multicore_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_multicore_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4398529594574569


If you use gensim and the following three variables, then you can visualize topics & keywords with the code below.

    lda_model:    this is an LDA model generated by gensim.models.ldamodel.LdaModel()
    id2word:      this is the dictionary term IDs from corpora.Dictionary()
    corpus:       this is the collection of "documents"


In [616]:
# Visualize topics-keywords
lda_model=lda_multicore_model
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.060352  0.005511  1       1        14.962501
3      0.048726 -0.054114  2       1        12.974576
8     -0.029337  0.073167  3       1        12.296324
2     -0.078260 -0.008342  4       1        11.871854
1     -0.059081  0.042843  5       1        10.337842
7     -0.077070 -0.007276  6       1        10.264083
5     -0.087577 -0.106061  7       1        9.337737 
0     -0.037998  0.110543  8       1        8.039827 
9      0.067704 -0.092236  9       1        6.573016 
6      0.192540  0.035966  10      1        3.342239 , topic_info=                  Term           Freq          Total Category  logprob  \
58     read             226018.000000  226018.000000  Default  30.0000   
474    book             139148.000000  139148.000000  Default  29.0000   
197    technology       110905.000000  110905.000000  Default  28.0000   
333    math             81286.000000   81286.000000   Default  27.0000   
393    seat             56463.000000   56463.000000   Default  26.0000   
1149   art              51648.000000   51648.000000   Default  25.0000   
193    science          59301.000000   59301.000000   Default  24.0000   
396    sit              45406.000000   45406.000000   Default  23.0000   
317    computer         41644.000000   41644.000000   Default  22.0000   
40     language         62012.000000   62012.000000   Default  21.0000   
441    chair            37220.000000   37220.000000   Default  20.0000   
1252   music            27636.000000   27636.000000   Default  19.0000   
387    play             51672.000000   51672.000000   Default  18.0000   
513    skill            121740.000000  121740.000000  Default  17.0000   
566    library          28809.000000   28809.000000   Default  16.0000   
16     english          44098.000000   44098.000000   Default  15.0000   
59     reader           29970.000000   29970.000000   Default  14.0000   
285    healthy          18827.000000   18827.000000   Default  13.0000   
185    move             54694.000000   54694.000000   Default  12.0000   
282    food             14271.000000   14271.000000   Default  11.0000   
537    game             35914.000000   35914.000000   Default  10.0000   
613    supply           52238.000000   52238.000000   Default  9.0000    
308    access           58067.000000   58067.000000   Default  8.0000    
990    chromebooks      20929.000000   20929.000000   Default  7.0000    
7      center           38958.000000   38958.000000   Default  6.0000    
944    desk             21612.000000   21612.000000   Default  5.0000    
833    stem             18071.000000   18071.000000   Default  4.0000    
88     child            100773.000000  100773.000000  Default  3.0000    
216    equipment        17256.000000   17256.000000   Default  2.0000    
376    flexible         19171.000000   19171.000000   Default  1.0000    
2198   sharpener        2646.189586    2649.427492    Topic1  -6.9709    
1846   flood            2273.120263    2280.759230    Topic1  -7.1229    
10470  brag             276.714835     277.703459     Topic1  -9.2288    
7542   copier           266.157586     267.108603     Topic1  -9.2677    
7970   normalcy         233.035248     233.955048     Topic1  -9.4006    
5856   rouge            212.481757     213.476761     Topic1  -9.4929    
4335   displace         331.312759     332.914809     Topic1  -9.0487    
9169   cello            172.536159     173.444105     Topic1  -9.7012    
14845  underwear        158.910191     159.817297     Topic1  -9.7834    
4379   aed              134.824338     135.782277     Topic1  -9.9478    
19544  scrapbook        124.497767     125.408503     Topic1  -10.0275   
9614   purifier         119.682159     120.596656     Topic1  -10.0669   
2195   misplace         245.516709     247.543464     Topic1  -9.3484    
11556  balm             110.012941     110.924767    

# PART 5:  Make an LDA topic model for the DESCRIPTIONS.

**Note: Part 5 is worth 5 points (the value of 5 individual problems).**

Using the same K (and any other hyperparameters from Part 4), recompute a model for Descriptions. Compare the two sets of results. Do they vary? How? Why? Explain what you find. 

In [583]:
df33['description'].head()

0    [apple, ipod, nano, gb, mp, player, generation, late, model, blue, apple, ipod, nano, gb, mp, player, generation, late, model, silver]        
1    [reebok, girl, fashion, dance, graphic, tshirt, dd, dark, heather, grey]                                                                      
2    [doodler, start, full, bundle]                                                                                                                
3    [ball, pg, poly, set, color, ball, playground, poly, set, kit, jumbo, gradestuff, pack, parachute, gripstarchute, recess, pack, grade, violet]
4    [crown, berkey, water, filter, black, pf, fluoride, filter]                                                                                   
Name: description, dtype: object

In [584]:

# Create Dictionary
id2word_description = corpora.Dictionary(df33['description'])

# Create Corpus
texts_description = df33['description']
# Term Document Frequency
corpus_description = [id2word_description.doc2bow(text) for text in texts_description]

# View unique id for each word in the essay
print(corpus_description[:1])


[[(0, 2), (1, 1), (2, 2), (3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (8, 2), (9, 2), (10, 1)]]


In [585]:
# 
import time
start_time = time.time()
##
## Build Multicore LDA
lda_multicore_model_d = gensim.models.ldamulticore.LdaMulticore(corpus_description, num_topics = 10, id2word=id2word_description)
# Saving trained model
lda_multicore_model_d.save('LDA_NYT_multicore_d2')
# Loading trained model
lda_multicore_model_d = gensim.models.ldamodel.LdaModel.load('LDA_NYT_multicore_d2')
## Print time taken to train the model
print("--- %s seconds ---" % (time.time() - start_time))




--- 36.78116583824158 seconds ---


In [586]:
# Print the Keyword in the 7 topics
print(lda_multicore_model_d.print_topics())
doc_lda_d = lda_multicore_model_d[corpus_description]

[(0, '0.020*"black" + 0.017*"color" + 0.015*"chair" + 0.014*"blue" + 0.014*"pack" + 0.013*"set" + 0.010*"staple" + 0.009*"seat" + 0.009*"assort" + 0.008*"pk"'), (1, '0.018*"pack" + 0.016*"ipad" + 0.015*"apple" + 0.015*"gb" + 0.012*"mini" + 0.012*"black" + 0.011*"inch" + 0.010*"case" + 0.008*"set" + 0.008*"color"'), (2, '0.037*"set" + 0.030*"ipad" + 0.025*"book" + 0.023*"level" + 0.022*"mini" + 0.017*"apple" + 0.017*"gr" + 0.013*"library" + 0.012*"case" + 0.012*"reader"'), (3, '0.053*"gb" + 0.029*"kid" + 0.024*"black" + 0.020*"chromebook" + 0.015*"memory" + 0.015*"fire" + 0.015*"celeron" + 0.014*"tablet" + 0.013*"chair" + 0.013*"intel"'), (4, '0.018*"pack" + 0.015*"inch" + 0.015*"color" + 0.014*"set" + 0.011*"card" + 0.010*"book" + 0.008*"sheet" + 0.008*"game" + 0.008*"resource" + 0.008*"box"'), (5, '0.027*"pack" + 0.023*"color" + 0.022*"paper" + 0.019*"set" + 0.011*"assort" + 0.009*"pencil" + 0.008*"white" + 0.008*"construction" + 0.008*"marker" + 0.008*"black"'), (6, '0.027*"set" + 0.

In [587]:
# Compute Perplexity
#print('\nPerplexity: ', lda_multicore_model_d.log_perplexity(corpus_description))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_multicore_model_d, texts=texts_description, dictionary=id2word_description, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.48648697594401924


In [588]:
# Visualize topics-keywords
lda_model=lda_multicore_model_d
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus_description, id2word_description)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.100717  0.080667  1       1        15.073770
2     -0.001975  0.063339  2       1        12.133259
9      0.030040 -0.149576  3       1        11.534207
8      0.007934 -0.136586  4       1        9.997926 
7     -0.104943 -0.014866  5       1        9.461154 
0     -0.017774  0.114138  6       1        9.021479 
3      0.278338  0.016898  7       1        8.919085 
1      0.041332  0.112320  8       1        8.675597 
6     -0.061313 -0.122129  9       1        7.632268 
4     -0.070921  0.035795  10      1        7.551255 , topic_info=                  Term           Freq          Total Category  logprob  \
2      gb               53782.000000   53782.000000   Default  30.0000   
91     ipad             35104.000000   35104.000000   Default  29.0000   
38     set              128012.000000  128012.000000  Default  28.0000   
0      apple            25622.000000   25622.000000   Default  27.0000   
262    mini             29501.000000   29501.000000   Default  26.0000   
189    level            25567.000000   25567.000000   Default  25.0000   
573    chromebook       14317.000000   14317.000000   Default  24.0000   
105    book             67072.000000   67072.000000   Default  23.0000   
68     chair            27323.000000   27323.000000   Default  22.0000   
32     pack             83360.000000   83360.000000   Default  21.0000   
398    paper            32760.000000   32760.000000   Default  20.0000   
716    oz               20600.000000   20600.000000   Default  19.0000   
168    gr               20297.000000   20297.000000   Default  18.0000   
26     color            66365.000000   66365.000000   Default  17.0000   
571    celeron          10781.000000   10781.000000   Default  16.0000   
860    memory           11865.000000   11865.000000   Default  15.0000   
51     kid              53899.000000   53899.000000   Default  14.0000   
384    subscription     10001.000000   10001.000000   Default  13.0000   
27     grade            23149.000000   23149.000000   Default  12.0000   
49     fire             12256.000000   12256.000000   Default  11.0000   
52     tablet           15309.000000   15309.000000   Default  10.0000   
41     black            63136.000000   63136.000000   Default  9.0000    
379    min              9068.000000    9068.000000    Default  8.0000    
586    intel            9411.000000    9411.000000    Default  7.0000    
409    bk               13666.000000   13666.000000   Default  6.0000    
174    wifi             15495.000000   15495.000000   Default  5.0000    
25     ball             32089.000000   32089.000000   Default  4.0000    
78     wobble           15858.000000   15858.000000   Default  3.0000    
591    ram              8623.000000    8623.000000    Default  2.0000    
65     sit              9052.000000    9052.000000    Default  1.0000    
18635  shurtech         187.972696     191.391296     Topic1  -8.6160    
11280  vivobook         30.869618      31.762816      Topic1  -10.4225   
14473  pretuned         52.760920      54.345696      Topic1  -9.8865    
16764  avanti           28.919383      29.811878      Topic1  -10.4878   
6636   sulphite         1223.818495    1264.240726    Topic1  -6.7426    
19202  marimba          27.503391      28.439460      Topic1  -10.5380   
4948   ta               4890.930752    5076.685387    Topic1  -5.3572    
18609  lokta            47.851025      49.671465      Topic1  -9.9842    
20853  sweatpant        72.586680      75.426037      Topic1  -9.5675    
18853  converse         37.374165      38.842779      Topic1  -10.2313   
8169   pn               154.903759     161.079688     Topic1  -8.8095    
13678  erlenmeyer       47.187116      49.109354      Topic1  -9.9982    
4019   flourish         33.011865      34.368221      Topic1  -10.3554   
14172  powertrc         19.971051      20.864003     

Description bubbles are overlapping more than essays' bubbles because names of techonologies/ sports clothes or books are different and should be divided into more than 10 topics
Essays could be separated because of similarity of words

# PART 6:  Use TextHero and help to improve it.

**Note: This is worth 5 points (the value of 5 individual problems).**

[TextHero](https://texthero.org/) is an opensource project developed by a student from the [TIS Lab of Prof. Younge](www.epfl.ch/labs/tis). Go to the [GIT repository for TextHero](https://github.com/jbesomi/texthero), install the package, review the documentation, and if you are impressed by the package - give it a star and tell others! (Not required)

Once you understand TextHero, then use the package to re-implement major portions of Part 3 of this assignment that you completed above.  

In [ ]:

!pip install texthero



In [595]:
import importlib
# Standard imports
import numpy  as np
import pandas as pd

import itertools
import random
import math  
import copy

from pprint import pprint  # nicer printing

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Other NLP
import re
import spacy
import nltk
from nltk.corpus import stopwords

# General Plotting
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
%matplotlib inline  
import seaborn as sns
sns.set(style="white")

# Special Plotting
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

# ignore some warnings 
import warnings
warnings.filterwarnings('ignore')

# Set the maximum number of rows displayed by pandas
pd.options.display.max_rows = 1000

# Set some CONSTANTS that will be used later
SEED    = 41  # base to generate a random number
SCORE   = 'roc_auc'
FIGSIZE = (16, 10)

In [589]:
import texthero as hero


In [590]:
data2=copy.copy(merged_df_textual)

In [591]:
#data2=data2.sample(n=100000, random_state=1)

In [592]:
data2.head(4)

,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
0,p036502,NV,Literacy & Language,Literacy,"Most of my kindergarten students come from low-income households and are considered \""at-risk\"". These kids walk to school alongside their parents and most have never been further than walking distance from their house. For 80% of my students, English is not their first language or the language spoken at home. \r\n\r\nWhile my kindergarten kids have many obstacles in front of them, they come to school each day excited and ready to learn. Most students started the year out never being in a school setting. At the start of the year many had never been exposed to letters. Each day they soak up more knowledge and try their hardest to succeed. They are highly motivated to learn new things every day. We are halfway through the year and they are starting to take off. They know know all letters, some sight words, numbers to 20, and a majority of their letter sounds because of their hard work and determination. I am excited to see the places we will go from here!I currently have a differentiated sight word center that we do daily during our literacy stations. The students have activities that relate to whatever sight word list they are on. This is one of their favorite station activities. I want to continue to provide the students with engaging ways to practice their sight words. \r\n\r\nI dream of having the students use QR readers to scan the sight words that they are struggling with and the Ipods reading the sight words with them. This would help so many of my students by giving them multiple exposures to the words. My students need someone who can go over these sight words daily and I can't always get around to everyone to practice their flashcards with them. With the Ipods they would still have a way to practice their sight words on a daily basis.",Apple - iPod nano� 16GB MP3 Player (8th Generation - Latest Model) - Blue Apple - iPod nano� 16GB MP3 Player (8th Generation - Latest Model) - Silver
2,p039565,GA,"Music & The Arts, Health & Sports","Performing Arts, Team Sports","Our elementary school is a culturally rich school, with a diverse population of 580 students, in Pre-K through sixth grade.\r\nOur Title I school population has 92% of students qualifying for free or reduced priced lunches and a high concentration of English Learners. We also serve two foster group homes for temporary and long-term placement of homeless children. \r\nWe do not see these statistics as road blocks. We see them as additions to our rich diversity. Together we will help students to develop to their fullest potential: Creative, problem-solving, compassionate adults.We strive to provide our diverse population of students with not only extra curricular activities, but an outlet for them to express themselves creatively.As a teacher, I have organized a dance club for lower elementary that meets once a week after school.\r\n\r\n This gives the girls something to look forward to, fosters the education of the whole child, and creates a social environment for our varied cultural student body. \r\n\r\nSince beginning our dance club, I have watched our girls who normally are introverts, bloom with excitement. They are also choreographing dances and productions in content areas during the school day.",Reebok Girls' Fashion Dance Graphic T-Shirt - Dd Dark Heather Grey - L
3,p233823,UT,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",Hello;\r\nMy name is Mrs. Brotherton. I teach 5th grade at Ascent Academy in Utah. We are a wonderful charter school that uses the students' interests to help them learn. We are always looking for wonderful teaching methods to help students. \r\nMy students are wonderful. I have several levels in my class. Our school is big on curriculum compacting and helping gifted kids move faster and struggling kids get the extra help they need. We would benefit so much from your 

In [593]:
from tqdm import tqdm

from tqdm._tqdm_notebook import tqdm_notebook
for column in tqdm(data2.columns):
    tqdm_notebook.pandas()
    data2[column] = data2[column].pipe(hero.clean)

100%|██████████| 6/6 [04:33<00:00, 45.58s/it]


In [594]:
data2.head()

,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
0,p036502,nv,literacy language,literacy,kindergarten students come low income households considered \ risk\ kids walk school alongside parents never walking distance house students english first language language spoken home \r\n\r\nwhile kindergarten kids many obstacles front come school day excited ready learn students started year never school setting start year many never exposed letters day soak knowledge try hardest succeed highly motivated learn new things every day halfway year starting take know know letters sight words numbers majority letter sounds hard work determination excited see places go currently differentiated sight word center daily literacy stations students activities relate whatever sight word list one favorite station activities want continue provide students engaging ways practice sight words \r\n\r\ni dream students use qr readers scan sight words struggling ipods reading sight words would help many students giving multiple exposures words students need someone go sight words daily always get around everyone practice flashcards ipods would still way practice sight words daily basis,apple ipod nano 16gb mp3 player 8th generation latest model blue apple ipod nano 16gb mp3 player 8th generation latest model silver
2,p039565,ga,music arts health sports,performing arts team sports,elementary school culturally rich school diverse population students pre k sixth grade \r\nour title school population students qualifying free reduced priced lunches high concentration english learners also serve two foster group homes temporary long term placement homeless children \r\nwe see statistics road blocks see additions rich diversity together help students develop fullest potential creative problem solving compassionate adults strive provide diverse population students extra curricular activities outlet express creatively teacher organized dance club lower elementary meets week school \r\n\r\n gives girls something look forward fosters education whole child creates social environment varied cultural student body \r\n\r\nsince beginning dance club watched girls normally introverts bloom excitement also choreographing dances productions content areas school day,reebok girls fashion dance graphic shirt dd dark heather grey l
3,p233823,ut,math science literacy language,applied sciences literature writing,hello \r\nmy name mrs brotherton teach 5th grade ascent academy utah wonderful charter school uses students interests help learn always looking wonderful teaching methods help students \r\nmy students wonderful several levels class school big curriculum compacting helping gifted kids move faster struggling kids get extra help need would benefit much donation every little bit helps us reach goal teach science 5th grade students clusters new students every weeks donation help one 5th grade class help several students mention students years come \r\nwe looking add 3doodler classroom would wonderful set 3doodler order help students achieve mission school big using technology help students express learned medium choice 3doodler class fast advanced learners able go ahead start project help students get 3doodler also allow one students find medium help learn retain knowledge,3doodler start full edu bundle
4,p185307,nc,health sports,health wellness,students greatest students socially economically disadvantaged inner city school limited activities pe physical education inside violence location school located physical activities students active within school good program \r\n\r\nmost students either african american hispanic students range enrolled kindergarten 8th grade since physical education important one success school students pe class monday friday every day proper equipment pe always possible asking help shelving \r\n\r\nthis project \ kid inspired\ want better fitness look back pe class said minutes day go extended day school year 

In [ ]:
#kernel died even though i tried 5 ttimes
"""data2['tfidf'] = (
    hero.tfidf(data2['essays']))"""

In [ ]:
#kernel died even though i tried 5 ttimes
"""
data2['pca'] = hero.pca(data2['tfidf'])
hero.scatterplot(
    df, 
    col='pca', 
    color='topic', 
    title="PCA BBC Sport news"
)"""

**Note: This is worth 5 points (the value of 5 individual problems).**

OpenSourcve packages rely on the community of users to help them grow and improve. Review the [contributing file](https://github.com/jbesomi/texthero/blob/master/CONTRIBUTING.md) for Text Hero and then identify a portion of the documentation that you feel could be improves. Edit/write 1 paragraph of documentation for the package that you believe would improve it. Copy that paragraph in below (into this notebook) so that it can be graded. And - if you think your contribution would truly help the project, please learn how to use git to suggest the change (a pull request) to the manager of the repository (Jonathan Besomi). 

Documentation for nmf

texthero.representation.nmf

nmf(s, n_components=2)

    Perform non-negative matrix factorization.

Find two non-negative matrices (W, H) whose product approximates the non-negative matrix X. 
This factorization can be used for example for dimensionality reduction, source separation or topic extraction.

Parameters

    s: Pandas Series
    n_components: Int. Default is 2.
        Number of components to keep. If n_components is not set or None, all components are kept.

Examples

import texthero as hero

import pandas as pd

s = pd.Series(["Sentence one", "Sentence two"])

custom_pipeline = [preprocessing.lowercase,
                   preprocessing.remove_whitespace]
s= hero.clean(s, custom_pipeline)
s_tf_idf = hero.tfidf(s)
pca=hero.nmf(s_tf_idf)

Documentation for pca

Example:
    
import texthero as hero

import pandas as pd

s = pd.Series(["Sentence one", "Sentence two"])

custom_pipeline = [preprocessing.lowercase,
                   preprocessing.remove_whitespace]
                   
s= hero.clean(s, custom_pipeline)

s_tf_idf = hero.tfidf(s)

pca=hero.pca(s_tf_idf)
